In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [3]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1420, with_units=False)
Hist_table


days,Entries
IntegerAndSexagesimal,Historical
1 ;,"00 ; 04,59"
2 ;,"00 ; 09,59"
3 ;,"00 ; 14,58"
4 ;,"00 ; 19,57"
5 ;,"00 ; 24,56"
6 ;,"00 ; 29,56"
7 ;,"00 ; 34,55"
8 ;,"00 ; 39,54"
9 ;,"00 ; 44,53"


In [4]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(31):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [5]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("0;4,59,15,27,7,23,50"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(31):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):
        Test_recomp[i][1]=daily_mean*Hist_table[i][0]%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

days,Entries
IntegerAndSexagesimal,Historical
1 ;,"00 ; 04,59"
2 ;,"00 ; 09,59"
3 ;,"00 ; 14,58"
4 ;,"00 ; 19,57"
5 ;,"00 ; 24,56"
6 ;,"00 ; 29,56"
7 ;,"00 ; 34,55"
8 ;,"00 ; 39,54"
9 ;,"00 ; 44,53"


In [6]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(0.0, nan)

In [7]:
B.astype(float).style.background_gradient(axis=None)

,Entries
days,
1 ;,0.000000
2 ;,0.000000
3 ;,0.000000
4 ;,0.000000
5 ;,0.000000
6 ;,0.000000
7 ;,0.000000
8 ;,0.000000
9 ;,0.000000


In [8]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Jupiter mean days.csv')
B.to_csv('Jupiter mean days.zip', index=False, compression=compression_opts)